## Enviroment

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import json
import psycopg2

Conexión a la base de datos mediante las credenciales definidas

In [13]:
credentials = "../credentials.json"
with open(credentials) as f:
    creds = json.load(f)
    
DB_USER = creds["user"]
DB_PASSWORD = creds["password"]
DB_HOST = creds["host"]
DB_PORT = creds["port"]
DB_NAME = creds["database"]

df = pd.read_csv("../data/candidates.csv", low_memory=False, encoding='ISO-8859-1', sep=";")
print("Datos del CSV cargados correctamente.")
print(df.head()) 

# Contar el número total de filas en el CSV
total_filas_csv = len(df)
print(f"El archivo CSV tiene un total de {total_filas_csv} filas.")

# Crear Base de Datos
creator_connection_string = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/postgres'
creator_engine = create_engine(creator_connection_string)

with creator_engine.connect() as connection:
    connection.execution_options(isolation_level="AUTOCOMMIT")
    result = connection.execute(
        text(f"SELECT 1 FROM pg_database WHERE datname = '{DB_NAME}'")
    )
    
    if not result.scalar():
        connection.execute(text(f"CREATE DATABASE {DB_NAME}"))
        print(f"Base de datos '{DB_NAME}' creada exitosamente!")
    else:
        print(f"La base de datos '{DB_NAME}' ya existe.")

# Crear conexión a la base de datos ya creada
connection_string = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(connection_string)

# Crear la tabla en la base de datos y cargar los datos
table_name = "candidates_wkshop_data"  # Nombre de la nueva tablan
df.to_sql(table_name, engine, if_exists='replace', index=False)

print(f"Tabla '{table_name}' creada y datos cargados en la base de datos '{DB_NAME}'.")

# Verificar los datos en la nueva tabla
with engine.connect() as connection:
    result = connection.execute(text(f"SELECT * FROM {table_name} LIMIT 5;"))
    print(f"Primeros 5 registros de la tabla '{table_name}':")
    for row in result:
        print(row)

Datos del CSV cargados correctamente.
   First Name   Last Name                      Email Application Date  \
0  Bernadette   Langworth        leonard91@yahoo.com       2021-02-26   
1      Camryn    Reynolds        zelda56@hotmail.com       2021-09-09   
2       Larue      Spinka   okey_schultz41@gmail.com       2020-04-14   
3        Arch      Spinka     elvera_kulas@yahoo.com       2020-10-01   
4       Larue  Altenwerth  minnie.gislason@gmail.com       2020-05-20   

   Country  YOE  Seniority                         Technology  \
0   Norway    2     Intern                      Data Engineer   
1   Panama   10     Intern                      Data Engineer   
2  Belarus    4  Mid-Level                     Client Success   
3  Eritrea   25    Trainee                          QA Manual   
4  Myanmar   13  Mid-Level  Social Media Community Management   

   Code Challenge Score  Technical Interview Score  
0                     3                          3  
1                     2   

In [14]:
# Contar el número total de filas en el DataFrame
total_csv_rows = len(df)
print(f"El archivo CSV tiene un total de {total_csv_rows} filas.")

# Verificar el número total de filas en la tabla
with engine.connect() as connection:
    result = connection.execute(text(f"SELECT COUNT(*) FROM {table_name};"))
    total_rows = result.scalar()  # Obtener el valor del conteo
    print(f"La tabla '{table_name}' tiene un total de {total_rows} filas.")

El archivo CSV tiene un total de 50000 filas.
La tabla 'candidates_wkshop_data' tiene un total de 50000 filas.


In [15]:
with engine.connect() as connection:
    # Consulta para obtener los primeros 20 registros de la tabla
    query = f"SELECT * FROM {table_name} LIMIT 20;"
    
    # Cargar los datos en un DataFrame de pandas
    df = pd.read_sql(query, connection)
    
    # Mostrar el DataFrame en formato tabular
    print(f"Primeros 20 registros de la tabla '{table_name}':")
    print(df)

Primeros 20 registros de la tabla 'candidates_wkshop_data':
    First Name    Last Name                      Email Application Date  \
0   Bernadette    Langworth        leonard91@yahoo.com       2021-02-26   
1       Camryn     Reynolds        zelda56@hotmail.com       2021-09-09   
2        Larue       Spinka   okey_schultz41@gmail.com       2020-04-14   
3         Arch       Spinka     elvera_kulas@yahoo.com       2020-10-01   
4        Larue   Altenwerth  minnie.gislason@gmail.com       2020-05-20   
5         Alec       Abbott   juanita_hansen@gmail.com       2019-08-17   
6      Allison       Jacobs   alba_rolfson27@yahoo.com       2018-05-18   
7          Nya       Skiles   madisen.zulauf@gmail.com       2021-12-09   
8         Mose        Lakin   dale_murazik@hotmail.com       2018-03-13   
9     Terrance        Zieme       dustin31@hotmail.com       2022-04-08   
10      Aiyana      Goodwin    vallie.damore@yahoo.com       2019-09-22   
11      Emilia      Waelchi      peter.g

In [ ]:
# Cargar el dataset
df = pd.read_csv("../data/candidates.csv", low_memory=False, encoding='ISO-8859-1', sep=";")

# Revisión inicial
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
print(df.info())  # Tipos de datos y valores no nulos
print(df.isnull().sum())  # Conteo de valores nulos por columna

Filas: 50000, Columnas: 1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 1 columns):
 #   Column                                                                                                                       Non-Null Count  Dtype 
---  ------                                                                                                                       --------------  ----- 
 0   First Name;Last Name;Email;Application Date;Country;YOE;Seniority;Technology;Code Challenge Score;Technical Interview Score  50000 non-null  object
dtypes: object(1)
memory usage: 390.8+ KB
None
First Name;Last Name;Email;Application Date;Country;YOE;Seniority;Technology;Code Challenge Score;Technical Interview Score    0
dtype: int64
